In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from helpers import *
from implementations import *

In [2]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y_train, X_train, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [4]:
X_train_zero, y_train_zero, X_train_one, y_train_one, X_train_many, y_train_many = split_by_jet_num(DATA_TRAIN_PATH, X_train, y_train)
X_test_zero, ids_test_zero, X_test_one, ids_test_one, X_test_many, ids_test_many = split_by_jet_num(DATA_TRAIN_PATH, X_test, ids_test)

In [5]:
def train_predict(X_train, y_train, X_test, max_iters=3000, degree=2, lambda_=0.01, gamma=0.01, imputable_th=0, encodable_th=1, verbose=True):
    tX_train, ty_train, tX_test, _, cont_features = preprocess(X_train, y_train, X_test, imputable_th=imputable_th, encodable_th=encodable_th, switch_encoding=True)
    tX_train_poly = build_poly(tX_train, degree=degree, cont_features=cont_features)
    weights, loss = reg_logistic_regression(ty_train, tX_train_poly, max_iters=max_iters, lambda_=lambda_, gamma=gamma, verbose=verbose)
    tX_test_poly = build_poly(tX_test, degree=degree, cont_features=cont_features)
    y_pred = predict_logistic(weights, tX_test_poly)
    y_pred = replace_values(y_pred, from_val=0, to_val=-1)
    return y_pred, weights, loss

In [6]:
degree = 2
lambda_ = 0.01
gamma = 0.01
max_iters = 3000

In [7]:
y_pred_zero, weights_zero, loss_zero = train_predict(X_train_zero, y_train_zero, X_test_zero, max_iters=max_iters, degree=degree, lambda_=lambda_, imputable_th=0.5)

Iteration = 0, accuracy = 27.355799545604675, loss = 4.873196092496017
Iteration = 10, accuracy = 29.419595047691494, loss = 4.179022439433003
Iteration = 20, accuracy = 32.112938256282966, loss = 3.5307493515776085
Iteration = 30, accuracy = 35.33374035410807, loss = 2.9554653891757177
Iteration = 40, accuracy = 39.20510844434658, loss = 2.471933032252379
Iteration = 50, accuracy = 43.218600182158475, loss = 2.0869651973936225
Iteration = 60, accuracy = 46.84975929058281, loss = 1.792437810426043
Iteration = 70, accuracy = 49.800326283866966, loss = 1.5678536064320647
Iteration = 80, accuracy = 52.171389108524416, loss = 1.3918634433277213
Iteration = 90, accuracy = 54.28622901924675, loss = 1.2524443198047621
Iteration = 100, accuracy = 56.14184340376127, loss = 1.1415184683864295
Iteration = 110, accuracy = 57.933402059792016, loss = 1.0528169781656658
Iteration = 120, accuracy = 59.49776305385686, loss = 0.9817662595270782
Iteration = 130, accuracy = 60.91199343428782, loss = 0.924

In [8]:
y_pred_one, weights_one, loss_one = train_predict(X_train_one, y_train_one, X_test_one, max_iters=max_iters, degree=degree, lambda_=lambda_, imputable_th=0.5)

Iteration = 0, accuracy = 35.73841947797379, loss = 7.117980078706898
Iteration = 10, accuracy = 35.77710719075621, loss = 6.381175450079116
Iteration = 20, accuracy = 36.0053646961725, loss = 5.613945176883278
Iteration = 30, accuracy = 36.668214175177965, loss = 4.834021723739728
Iteration = 40, accuracy = 38.21701227690086, loss = 4.067727630503282
Iteration = 50, accuracy = 40.4763747033942, loss = 3.353485689659376
Iteration = 60, accuracy = 43.497885071701226, loss = 2.7298565810200697
Iteration = 70, accuracy = 46.92561642422366, loss = 2.219677830034287
Iteration = 80, accuracy = 50.17925306922522, loss = 1.8283380728348302
Iteration = 90, accuracy = 53.363251831218406, loss = 1.5460220453499545
Iteration = 100, accuracy = 55.79412978438049, loss = 1.3470902878271502
Iteration = 110, accuracy = 57.66403590219746, loss = 1.2050737017061655
Iteration = 120, accuracy = 59.243784174146285, loss = 1.1038578342219219
Iteration = 130, accuracy = 60.48952852574022, loss = 1.03245548196

In [9]:
y_pred_many, weights_many, loss_many = train_predict(X_train_many, y_train_many, X_test_many, max_iters=max_iters, degree=degree, lambda_=lambda_, imputable_th=0.5)

Iteration = 0, accuracy = 44.75276732420771, loss = 7.463962918553235
Iteration = 10, accuracy = 44.75276732420771, loss = 6.76497306533056
Iteration = 20, accuracy = 44.763795266255876, loss = 5.999796782206942
Iteration = 30, accuracy = 44.863046744689356, loss = 5.181476801591986
Iteration = 40, accuracy = 45.31932784693217, loss = 4.3375341144432955
Iteration = 50, accuracy = 46.67300773334436, loss = 3.5238261422359143
Iteration = 60, accuracy = 48.61806101208938, loss = 2.8127262234623656
Iteration = 70, accuracy = 50.83192037825841, loss = 2.2469706893787076
Iteration = 80, accuracy = 52.936878816701814, loss = 1.8293049709843736
Iteration = 90, accuracy = 54.93982879119971, loss = 1.5384764890333331
Iteration = 100, accuracy = 56.778738127731145, loss = 1.341011821729397
Iteration = 110, accuracy = 58.30610810140193, loss = 1.205505013260816
Iteration = 120, accuracy = 59.67495140813035, loss = 1.1095992005247624
Iteration = 130, accuracy = 60.76809616365466, loss = 1.039210445

In [10]:
y_pred = np.vstack([y_pred_zero, y_pred_one, y_pred_many])
ids_test = np.hstack([ids_test_zero, ids_test_one, ids_test_many])

In [11]:
from datetime import datetime
method = 'reg_logistic_regression_by_jet'
time = datetime.now().strftime('%Y%m%dH%H%M%S')
OUTPUT_PATH = f'../submissions/submission_{method}_{time}'
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [15]:
param_grid = {
    'lambda_': np.logspace(-4, 0, 5),
    'degree': list(range(1, 4)),
    'max_iters': 100,
    'gamma': [0.01, 0.05, 0.1],
    'cont_features': [cont_features]
}
metrics, params = logistic_regression_cv(ty_train, tX_train, param_grid=param_grid)

In [16]:
metrics

{'loss': 0.4160482322829052,
 'accuracy': 81.97132616487455,
 'f1_score': 0.7979288473644017}

In [17]:
params

{'lambda_': 0.001,
 'degree': 3,
 'gamma': 0.1,
 'max_iters': 1000,
 'cont_features': (1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29)}

In [ ]:
tX_train, ty_train, tX_test, _, cont_features = preprocess(X_train_many, y_train_many, X_test_many, imputable_th=0, encodable_th=1, switch_encoding=True)
tX_train_poly = build_poly(tX_train, degree=degree, cont_features=cont_features)
ty_train_pred = predict_logistic(weights_many, tX_train_poly)
train_accuracy = compute_accuracy(ty_train, ty_train_pred)
train_f1 = compute_f1(ty_train, ty_train_pred)
train_accuracy, train_f1

In [ ]:
features = read_feature_names(DATA_TRAIN_PATH)
one_jet_features = [f for f in features if f not in ['PRI_jet_num', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet',
                                                     'DER_lep_eta_centrality', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi']]

In [ ]:
features = read_feature_names(DATA_TRAIN_PATH)
many_jet_features = [f for f in features if f not in ['PRI_jet_num']]

In [ ]:
def plot_features_distrib(tX):
    plt.figure(figsize=(15,30))
    for i in range(tX.shape[1]):
        plt.subplot(10,3,i+1)
        plt.hist(tX[:,i], bins=50)
        plt.yscale('log')
        plt.title("Feature {}: {}".format(i, many_jet_features[i]))
    plt.tight_layout()
    plt.show();
plot_features_distrib(X_train_many[(ty_train_pred != ty_train).reshape((-1,))])

In [ ]:
tX_val_poly = build_poly(tX_test, degree=params['degree'], cont_features=cont_features)
ty_val_pred = predict_logistic(weights, tX_val_poly)
val_accuracy = compute_accuracy(ty_test, ty_val_pred)
val_f1 = compute_f1(ty_test, ty_val_pred)
val_accuracy, val_f1

In [ ]:
val_tmp = np.hstack([X_train_val, ty_val_pred, ty_test])

In [ ]:
val_tmp_mis = val_tmp[val_tmp[:, -1] != val_tmp[:, -2]]

In [ ]:
tval_tmp = replace_values(val_tmp_mis, -999, np.nan)
col_nan_ratio = compute_nan_ratio(tval_tmp)

In [ ]:
col_nan_ratio

In [ ]:
def plot_features_distrib(tX):
    plt.figure(figsize=(15,30))
    for i in range(tX.shape[1]):
        plt.subplot(10,3,i+1)
        plt.hist(tX[:,i], bins=50);
        # plt.title("Feature {}: {}".format(i, labels[i]))
    plt.tight_layout()
    plt.show();
plot_features_distrib(tval_tmp[:, :-2])

In [ ]:
tX_test.shape

In [ ]:
tX_test_poly = build_poly(tX_test, params['degree'], cont_features)

In [ ]:
tX_test_poly.shape

In [ ]:
from datetime import datetime
method = 'reg_logistic_regression_by_jet'
time = datetime.now().strftime('%Y%m%dH%H%M%S')
OUTPUT_PATH = f'../submissions/submission_{method}_{time}'
# y_pred = predict_logistic(weights, tX_test_poly)
# y_pred = replace_values(y_pred, from_val=0, to_val=-1)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)